# <font color='red'>1. Setup and Installation</font>

## <font color='green'>1.1 Download repo + requirements</font>

In [ ]:
%cd /content
!git clone https://github.com/metaltiger775/geoguessr-ai-colab
%cd geoguessr-ai-colab

/content
Cloning into 'geoguessr-ai-colab'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 279 (delta 36), reused 2 (delta 0), pack-reused 219
Receiving objects: 100% (279/279), 118.13 KiB | 14.77 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/geoguessr-ai-colab


In [ ]:
!pip install kaggle
!pip install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


##  <font color='green'>1.2 Download & Unzip premade dataset</font> **``OPTION 1``**

### <font color='purple'>1.2.1 Create kaggle.json</font>

In [ ]:
#@markdown **Manually save your API token in .kaggle/kaggle.json**
!sudo rm -rf ~/.kaggle
!sudo rm -f ~/.kaggle/kaggle.json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

username = "YOUR-USERNAME-HERE" #@param {type:"string"}
key = "YOUR-API-KEY-HERE" #@param {type:"string"}
api_token = {"username":username,"key":key}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

### <font color='purple'>1.2.2 Download dataset</font>

In [ ]:
!mkdir /content/geoguessr-ai-colab/images
!kaggle datasets download stelath/city-street-view-dataset -p /content/geoguessr-ai-colab/images

100% 7.24G/7.24G [06:06<00:00, 23.8MB/s]
100% 7.24G/7.24G [06:06<00:00, 21.2MB/s]


### <font color='purple'>1.2.3 Unzip dataset</font>

In [ ]:
!unzip /content/geoguessr-ai-colab/images/city-street-view-dataset.zip -d /content/geoguessr-ai-colab/images

Streaming output truncated to the last 5000 lines.
  inflating: /content/geoguessr-ai-colab/images/street_view_5499.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_55.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_550.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5500.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5501.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5502.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5503.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5504.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5505.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5506.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5507.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5508.jpg  
  inflating: /content/geoguessr-ai-colab/images/street_view_5509.jpg  
  inflating: /content/geogues

In [ ]:
!sudo rm -rf /content/geoguessr-ai-colab/images/city-street-view-dataset.zip

## <font color='green'>1.3 Make your own dataset</font> **``OPTION 2``**

In [ ]:
#@markdown **Enter your Google Street View API key:**
key = "" #@param {type:"string"}
#@markdown *How many images do you want to get?*
num_images = 25000 #@param {type:"integer"}
#@markdown *The folder full of addresses per city to read and extract GPS coordinates from*
folder = "geoguessr_dataset/" #@param {type:"string"}
#@markdown *The output folder for where the images will be stored*

!python -m get_images.py --key $key --icount $num_images --cities $folder

## <font color='green'>1.4 Dataset builder</font>

In [ ]:
!python -m dataset_builder_multi_label --file images/picture_coords.csv --images images/ --output geoguessr_dataset/

100% 48953/48953 [06:22<00:00, 128.10it/s]
Train Files: 44028
Val Files: 4925


# <font color='red'>2. Training</font>

## <font color='green'>2.1 Show help</font>

In [ ]:
!python -m main -h

2023-12-20 22:29:05.172774: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 22:29:05.172897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 22:29:05.298376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 22:29:08.006149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: main.py [-h] [-a ARCH] [-j N] [--epochs N] [--checkpoint-step N] [-b N] [--lr LR] [-p N]
               [--resume PATH] [-e]
               DIR

PyTorch GeoGuessr AI Training

positional arguments:
  DIR                   path to data

## <font color='green'>2.2 Train</font>

In [ ]:
#@title Settings for training

#@markdown ---
save_to_drive = True #@param {type:"boolean"}
arch = "wide_resnet50_2" #@param {type:"string"}['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny','densenet121', 'densenet161', 'densenet169', 'densenet201','efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3','efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7','efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s','get_model', 'get_model_builder', 'get_model_weights', 'get_weight','googlenet', 'inception_v3', 'list_models', 'maxvit_t','mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3','mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small','regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf','regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf','regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf','regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf','resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50','resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d','shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0','squeezenet1_0', 'squeezenet1_1','swin_b', 'swin_s', 'swin_t', 'swin_v2_b', 'swin_v2_s', 'swin_v2_t','vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn','vit_b_16', 'vit_b_32', 'vit_h_14', 'vit_l_16', 'vit_l_32','wide_resnet101_2', 'wide_resnet50_2']
#4
workers = 4 #@param {type:"integer"}
#90
epochs = 90 #@param {type:"number"}
#@markdown How often to save the model
checkpoint_step = 1 #@param {type:"integer"}
#65
batch_size = 65 #@param {type:"integer"}
learning_rate = 0.001 #@param {type:"number"}
#@markdown How often to print progress
print_freq = 10 #@param {type:"integer"}
#@markdown ---

from google.colab import drive
drive.mount('/content/drive')

# Run training
!python -m main geoguessr_dataset/ -a $arch -b $batch_size --lr $learning_rate -j $workers --checkpoint-step $checkpoint_step --epochs $epochs --print-freq $print_freq --save-to-drive $save_to_drive


## Base training settings
#!python -m main geoguessr_dataset/ -a wide_resnet50_2 -b 16 --lr 0.0001 -j 6 --checkpoint-step 1






# <font color='red'>3. Other</font>

In [ ]:
#@title Reset model folder (dangerous!)

!sudo rm -rf /content/geoguessr-ai-colab/models
!mkdir /content/geoguessr-ai-colab/models

In [ ]:
#@title Remove dataset (dangerous!)

!sudo rm -rf /content/geoguessr-ai-colab/geoguessr_dataset

In [ ]:
#@title Remove Google Drive model folder (dangerous!)

!sudo rm -rf /content/drive/MyDrive/GeoGuessr
!sudo mkdir /content/drive/MyDrive/GeoGuessr
!sudo mkdir /content/drive/MyDrive/GeoGuessr/models